# Predict Conversions from Quotes

**Goal**: Build a model to predict whether a quote will convert to a purchase.

**Data Sources**
* MySQL records of quote details and outcome
* Paypal records of flooring samples purchases
* Mailchimp subscriber list

In [1]:
import bd_mailchimp
import bd_paypal
import bd_mysql

import pandas as pd
import numpy as np

## Load data

In [2]:
# Load mailchimp data
csv_path = '/Users/lindsay/Documents/Data Science/BrazilianDirect/csv/mailchimp/members_export_21_march_2016.csv'
mail, first_email = bd_mailchimp.process_mailchimp(csv_path)

In [3]:
# Load paypal data
csv_dir = '/Users/lindsay/Documents/Data Science/BrazilianDirect/csv/paypal/'
paypal, first_sample = bd_paypal.process_paypal(csv_dir)

In [4]:
# Load MySQL data
config_path = '/Users/lindsay/Documents/Data Science/BrazilianDirect/cfg/mysql.cfg'
con = bd_mysql.connect_bd_mysql(config_path)
df = bd_mysql.download_quote_data(con)
df = bd_mysql.pre_process_mysql(df)

## Join data

In [5]:
# merge mysql & mail chimp
df_all = pd.merge(df, mail, how='left', on='email')

# add paypal
df_all = pd.merge(df_all, paypal, how='left', on='email')

In [6]:
# Replace nan with 0
df_all['mail_chimp'] = df_all['mail_chimp'].fillna(value=0)
df_all['samples'] = df_all['samples'].fillna(value=0)

## Filter out quotes before mail chimp & samples

In [7]:
earliest_date = pd.datetime.date(max(first_email, first_sample))
df_all = df_all.loc[df_all['date_created'] >= earliest_date, :]
df_all.shape

(20821, 26)

## Drop columns that won't be used for predictions

* `quote_id`: unique identifier
* `email`: nearly unique identifier (not many repeat customers)
* `date_created`: too fine grained to use
* `days_until_needed`: transformed into a binned variable
* `ship_state`: transformed into a grouped variable for regions
* `install_subfloor`: too many missing values
* `sq_ft`: transformed into a binned variable
* `milling`: milling is perfectly correlated with `finish` (unfinished = square edge, prefinished = micro bevel)
* `year`: interested only in monthly seasonality
* `state_division`: will use the regional divisions, which have fewer categories

In [8]:
drop_cols = ['quote_id',
             'email',
             'date_created',
             'days_until_needed',
             'ship_state',
             'install_subfloor',
             'sq_ft',
             'milling',
             'year',
             'state_division']
df_all = df_all.drop(drop_cols, axis=1)

## Prepare data for sklearn

### Extract variables from data fame

In [9]:
df_all.columns

Index([u'phone_provided', u'employee_id', u'cust_price', u'retail_price',
       u'common_name', u'finish', u'grade', u'width', u'construction',
       u'days_until_needed_bin', u'sq_ft_bin', u'converted', u'month',
       u'state_region', u'mail_chimp', u'samples'],
      dtype='object')

In [10]:
# column types
dependent_column = ['converted']
cat_columns = ['phone_provided',
               'employee_id',
               'common_name',
               'finish',
               'grade', 
               'width', 
               'construction',
               'days_until_needed_bin',
               'sq_ft_bin',
               'month', 
               'state_region',
               'mail_chimp',
               'samples']
meas_columns = ['cust_price',
                'retail_price']

In [11]:
# Extract variables
y = df_all.as_matrix(columns=dependent_column)
x_cat = df_all.as_matrix(columns=cat_columns)
x_meas = df_all.as_matrix(columns=meas_columns)

### Transform categorical variables to 0-indexed integers

In [12]:
from sklearn.preprocessing import LabelEncoder
labels = {}
for i, col in enumerate(cat_columns):
    
    # Extract the column values and convert to 0-indexed integers
    x = x_cat[:, i]
    le = LabelEncoder().fit(x)
    x_cat[:, i] = le.transform(x)
    
    # Store label data
    labels[col] = le

//anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
//anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


### Impute missing values
We'll impute with the median value for the numerical columns that are missing data (`cust_price`) and impute with the mode value for missing categorical data (`days_until_needed_bin` and `sq_ft_bin`).

In [15]:
from sklearn.preprocessing import Imputer

In [16]:
# Examine number of missing values in each column
for col in df_all.columns:
    print col, ': ', df_all[col].isnull().sum()

phone_provided :  0
employee_id :  0
cust_price :  842
retail_price :  688
common_name :  0
finish :  0
grade :  0
width :  0
construction :  0
days_until_needed_bin :  10174
sq_ft_bin :  3
converted :  0
month :  0
state_region :  0
mail_chimp :  0
samples :  0


In [17]:
# Impute measured variables
imp_meas = Imputer(missing_values=np.nan, strategy='median', axis=0)

# Fit imputer
imp_meas.fit(x_meas)

# Impute values
x_meas_imp = imp_meas.transform(x_meas)

In [21]:
# Impute categorical variables
imp_cat = Imputer(missing_values=np.nan, strategy='most_frequent', axis=0)

# Fit imputer
imp_cat.fit(x_cat)

# Impute values
x_cat_imp = imp_cat.transform(x_cat)

### Perform one-hot encoding on categorical variables

In [23]:
from sklearn.preprocessing import OneHotEncoder

In [24]:
# Use one-hot encoding to transform categorical variable into
# multiple binary variables
enc = OneHotEncoder()
enc.fit(x_cat_imp)

# Apply one-hot encoding
x_cat_imp = enc.transform(x_cat_imp).toarray()

In [33]:
# Create a list so we know what the one-hot encoded columns refer to
one_hot_columns = []
for col in cat_columns:
    features = list(labels[col].classes_)
    for f in features:
        one_hot_columns.append(col + '_' + str(f))

### Scale continuous features

In [35]:
from sklearn.preprocessing import scale
x_meas_imp = scale(x_meas_imp)

### Concatenate categorical and continuous features

In [37]:
x = np.concatenate((x_cat_imp, x_meas_imp), axis=1)
x_names = one_hot_columns + meas_columns